In [235]:
import pandas as pd
import numpy as np
import scipy.stats as si
import quantstats

from Dates import from_excel_date
from VolCarry import VolCarryBacktester

In [318]:
currency = 'MXN'
tenor = '1M'
tc = 0.2e-2

In [319]:
spot = pd.read_excel(fr'data/FX.xlsx', sheet_name='Spot', index_col=0)[currency]
spot.drop('Date',inplace=True)
spot.index.rename('Date', inplace=True)
spot.index = spot.index.map(from_excel_date)
spot.index = [d.date() for d in spot.index]
spot.dropna(inplace=True)
spot

2000-01-03      9.505
2000-01-04     9.5713
2000-01-05      9.571
2000-01-06       9.58
2000-01-07      9.565
               ...   
2025-12-26    17.9115
2025-12-29    17.9794
2025-12-30    17.9952
2025-12-31     18.008
2026-01-01    17.9986
Name: MXN, Length: 6781, dtype: object

In [320]:
iv = pd.read_excel(fr'data/FX.xlsx', sheet_name=f'{currency} Vol', index_col=0)[tenor]
iv.drop('Date',inplace=True)
iv.index.rename('Date', inplace=True)
iv.index = iv.index.map(from_excel_date)
iv.index = [d.date() for d in iv.index]
iv.dropna(inplace=True)
iv

2003-10-01     12.65
2003-10-02     12.65
2003-10-16      12.5
2003-10-17     12.75
2003-10-20     13.15
               ...  
2025-12-26    12.923
2025-12-29     13.32
2025-12-30    13.142
2025-12-31    13.017
2026-01-01    13.355
Name: 1M, Length: 5771, dtype: object

In [321]:
# Match index for spot and iv
index_ = spot.index.intersection(iv.index)
spot = spot[index_]
iv = iv[index_]

In [362]:
import numpy as np
import pandas as pd
from scipy.stats import norm

class FXStraddleBacktester:
    def __init__(self, spot_series, implied_vol, r_dom, r_for, tenor_days=21, transaction_cost_pct=0.0005):
        """
        :param spot_series: Pandas Series of Spot Prices
        :param implied_vol: Annualized Implied Vol
        :param r_dom: Domestic Risk-Free Rate
        :param r_for: Foreign Risk-Free Rate
        :param tenor_days: Days to maturity
        :param transaction_cost_pct: The cost to trade as a % of notional (Half-Spread).
                                     e.g., 0.0002 = 2 basis points cost (approx 4bps wide spread).
        """
        self.spots = spot_series
        self.iv = implied_vol
        self.r_d = r_dom
        self.r_f = r_for
        self.T_total = tenor_days / 252.0
        self.dt = 1 / 252.0
        self.tc = transaction_cost_pct  # NEW: Transaction Cost parameter
        
        self.K = self.spots.iloc[0]

    def garman_kohlhagen(self, S, K, T, r_d, r_f, sigma):
        if T <= 0:
            return max(S - K, 0), max(K - S, 0), 1.0 if S > K else 0.0, -1.0 if K > S else 0.0

        d1 = (np.log(S / K) + (r_d - r_f + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)

        df_d = np.exp(-r_d * T)
        df_f = np.exp(-r_f * T)

        c_price = S * df_f * norm.cdf(d1) - K * df_d * norm.cdf(d2)
        c_delta = df_f * norm.cdf(d1) 
        
        p_price = K * df_d * norm.cdf(-d2) - S * df_f * norm.cdf(-d1)
        p_delta = -df_f * norm.cdf(-d1)

        return c_price, p_price, c_delta, p_delta

    def run_backtest(self):
        dates = self.spots.index
        n_days = len(dates)
        
        cash_balance = 0.0
        hedge_units = 0.0 
        
        results = []

        # --- INCEPTION ---
        S0 = self.spots.iloc[0]
        c_price, p_price, c_delta, p_delta = self.garman_kohlhagen(
            S0, self.K, self.T_total, self.r_d, self.r_f, self.iv
        )
        
        # Sell Straddle
        mid_premium = c_price + p_price
        # NEW: Apply transaction cost to premium (Selling at Bid)
        # Cost is applied to the Notional value of the option structure, or strictly on premium. 
        # Standard convention: Cost on the Notional underlying equivalent if using a generic 'tc' parameter.
        # But for simplicity here, let's assume 'tc' is applied to the premium transaction value or spot value.
        # Let's apply it to the Premium for the option leg (conservative estimate of spread).
        premium_collected = mid_premium * (1 - self.tc) 
        
        cash_balance += premium_collected
        
        # Initial Hedge
        target_hedge = -1 * -(c_delta + p_delta)
        trade_units = target_hedge
        
        # NEW: Calculate Trading Cost on Spot
        trade_value = trade_units * S0
        spot_cost = abs(trade_units) * S0 * self.tc
        
        cash_balance -= (trade_value + spot_cost) # Deduct cost from cash
        hedge_units = target_hedge
        
        results.append({
            'date': dates[0],
            'spot': S0,
            'portfolio_value': premium_collected, 
            'hedge_units': hedge_units,
            'cash': cash_balance,
            'transaction_costs': spot_cost,
            'pnl_daily': 0.0
        })

        # --- DAILY LOOP ---
        for i in range(1, n_days):
            S_t = self.spots.iloc[i]
            T_remaining = self.T_total - (i * self.dt)
            
            # 1. Accrue Interest
            interest_cash = cash_balance * (self.r_d * self.dt)
            interest_foreign = (hedge_units * S_t) * (self.r_f * self.dt)
            total_carry = interest_cash + interest_foreign
            cash_balance += total_carry

            # 2. Mark to Market Options
            c_val, p_val, c_del, p_del = self.garman_kohlhagen(
                S_t, self.K, max(T_remaining, 1e-9), self.r_d, self.r_f, self.iv
            )
            liability_value = c_val + p_val
            
            # 3. Re-Hedge
            straddle_delta = -(c_del + p_del)
            new_target_hedge = -straddle_delta
            
            trade_units = new_target_hedge - hedge_units
            
            # NEW: Transaction Costs Logic
            trade_principal = trade_units * S_t
            trade_fee = abs(trade_units) * S_t * self.tc # Cost is proportional to volume traded
            
            cash_balance -= (trade_principal + trade_fee)
            hedge_units = new_target_hedge
            
            # 4. Total Value
            portfolio_nav = cash_balance + (hedge_units * S_t) - liability_value
            
            pnl = portfolio_nav - results[-1]['portfolio_value']

            results.append({
                'date': dates[i],
                'spot': S_t,
                'portfolio_value': portfolio_nav,
                'hedge_units': hedge_units,
                'cash': cash_balance,
                'transaction_costs': trade_fee, # Track this specifically
                'pnl_daily': pnl
            })

        return pd.DataFrame(results).set_index('date')


In [ ]:
# --- EXAMPLE COMPARISON ---
# Let's see how much costs eat into the P&L

# Dummy Data
np.random.seed(42)
days = 22
spot_start = 20.00
returns = np.random.normal(0, 0.12/np.sqrt(252), days) # 12% Realized Vol
spot_series = pd.Series(spot_start * (1 + np.cumsum(returns)), index=pd.date_range('2024-01-01', periods=days))

iv = 0.14 # 14% Implied Vol

# Run 1: Zero Costs
bt_zero = FXStraddleBacktester(spot_series, iv, 0.05, 0.11, transaction_cost_pct=0.0)
res_zero = bt_zero.run_backtest()

# Run 2: 5bps Costs (typical for emerging markets institutional execution)
bt_cost = FXStraddleBacktester(spot_series, iv, 0.05, 0.11, transaction_cost_pct=0.0005)
res_cost = bt_cost.run_backtest()

print(f"P&L (Zero Cost): {res_zero['portfolio_value'].iloc[-1]:.4f}")
print(f"P&L (With Cost): {res_cost['portfolio_value'].iloc[-1]:.4f}")

total_fees = res_cost['transaction_costs'].sum()
print(f"Total Fees Paid: {total_fees:.4f}")

In [341]:
spot.iloc[0:21]

2003-10-01    11.0275
2003-10-02    11.1645
2003-10-16     11.287
2003-10-17    11.2233
2003-10-20    11.1213
2003-10-21    11.1488
2003-10-22    11.2443
2003-10-23     11.167
2003-10-24    11.1548
2003-10-27     11.096
2003-10-28    11.0595
2003-10-29     11.091
2003-10-30    11.0425
2003-10-31    11.0175
2003-11-03     10.985
2003-11-04    10.9963
2003-11-05    10.9775
2003-11-06    10.9538
2003-11-07    10.9875
2003-11-10     10.945
2003-11-11    10.9778
Name: MXN, dtype: object

In [340]:
iv

2003-10-01     12.65
2003-10-02     12.65
2003-10-16      12.5
2003-10-17     12.75
2003-10-20     13.15
               ...  
2025-12-26    12.923
2025-12-29     13.32
2025-12-30    13.142
2025-12-31    13.017
2026-01-01    13.355
Name: 1M, Length: 5768, dtype: object

In [366]:
# Run 2: 5bps Costs (typical for emerging markets institutional execution)
bt_cost = FXStraddleBacktester(spot.iloc[0:21], iv.iloc[0], 0.04, 0.07, transaction_cost_pct=0.0005)
res = bt_cost.run_backtest()

In [368]:
res

,spot,portfolio_value,hedge_units,cash,transaction_costs,pnl_daily
date,,,,,,
2003-10-01,11.0275,20.453907,0.926606,10.230651,0.005109,0.000000
2003-10-02,11.1645,0.137066,0.920556,10.302664,0.000034,-20.316841
2003-10-16,11.2870,0.307237,0.913795,10.383456,0.000038,0.170171
2003-10-17,11.2233,0.495938,0.905386,10.482279,0.000047,0.188700
2003-10-20,11.1213,0.705085,0.895741,10.593951,0.000054,0.209148
2003-10-21,11.1488,0.938669,0.885696,10.710337,0.000056,0.233584
2003-10-22,11.2443,1.200756,0.874934,10.835757,0.000061,0.262087
2003-10-23,11.1670,1.493211,0.861591,10.989120,0.000075,0.292455
2003-10-24,11.1548,1.821297,0.847080,11.155316,0.000081,0.328086


array([20.45390746,  0.13706603,  0.30723737,  0.49593754,  0.70508504,
        0.93866919,  1.20075584,  1.49321104,  1.82129691,  2.18955276,
        2.60466785,  3.07582084,  3.61107149,  4.22322516,  4.92798992,
        5.74839203,  6.71368523,  7.86873454,  9.29081938, 11.1144105 ,
       13.69385105])